# Lab 2: Localization of Wireless Cameras

### Important Notes:

-The xloc and yloc are all flipped in the data given to us

-Units are in inches.

-We need to localize 3 devices with the following MAC addresses:
    1. f8:cf:c5:97:e0:9e
    2. ec:d0:9f:db:e8:1f
    3. 80:e6:50:1b:a7:80
    4. Ground truth of device with MAC 44:91:60:d3:d6:94 is (-22, 162)
  Plan:
    1. We sort the all datasets into 4 sub-datasets for each MAC.
    2. The algorithm will use the log distance path loss model equation shown in detail in Lecture 7 Slides.
    3. Test the algorithm for the ground truth device.
    4. Extract location of other 3 devices.

In [1]:
import numpy as np, pandas as pd, json, os, matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

## 1. Take all datasets and sort them into 4 tables, one for each MAC address

In [2]:
cf = []
d0 = []
e6 = [] #initializaing empty arrays for each mac address
gt = [] #ground-truth device
with open('car_traces.txt','r') as f:
    files = list(filter(lambda x:x!='',f.read().split('\n')))
    fileind = {}
    for file in files:
        inds = file.split()
        if inds[0] in fileind:
            fileind[inds[0]].append(inds[1])
        else:
            fileind[inds[0]] = [inds[1]]
desiredfiles = fileind['eea7']
for file in desiredfiles: #takes data from every file in folder
    filename = os.fsdecode(file)
    path = 'final_lab2_data/' + filename
    with open(path, 'r') as f:
        data = (json.loads(f.read().replace('\'','\"')))
    for x in range(0, len(data)):
        entry = data[x]
        if entry['mac'] == 'f8:cf:c5:97:e0:9e':
            cf.append({'loc_x': entry['loc_y'], 'loc_y': entry['loc_x'], 'rss': entry['rss']})
        elif entry['mac'] == 'ec:d0:9f:db:e8:1f':
            d0.append({'loc_x': entry['loc_y'], 'loc_y': entry['loc_x'], 'rss': entry['rss']})
        elif entry['mac'] == '80:e6:50:1b:a7:80':
            e6.append({'loc_x': entry['loc_y'], 'loc_y': entry['loc_x'], 'rss': entry['rss']})
        elif entry['mac'] == '44:91:60:d3:d6:94':
            gt.append({'loc_x': entry['loc_y'], 'loc_y': entry['loc_x'], 'rss': entry['rss']})
cfdf = pd.DataFrame(cf).apply(pd.to_numeric)
d0df = pd.DataFrame(d0).apply(pd.to_numeric)
e6df = pd.DataFrame(e6).apply(pd.to_numeric)
gtdf = pd.DataFrame(gt).apply(pd.to_numeric)

## 2. Fitting given missing parameters of log-distance path loss model

The log-distance path loss equation:

$P_{RX}[dBm]=C-\gamma*log(d(TX,RX))$

We will be using the methodology detailed in Xu et al.'s _RSS-Based Source Localization When Path-Loss Model Parameters are Unknown_, which takes a search approach with different tested ratios of power to find the most accurate $\gamma$ for the set within $.05$. The approach will also let us calculate $(x_{TX}, y_{TX})$ directly without having to find $C$.

For the sake of reducing noise from conflicting distributions we will only be using the trace data from one car, eea7.

In [3]:
def findTrans(df):
    optgam,optsum,optx,opty = 0,None,0,0
    x1 = df.loc_x.iloc[0]
    y1 = df.loc_y.iloc[0]
    xj = df.loc_x.iloc[1:]
    yj = df.loc_y.iloc[1:]
    for i in np.arange(1,5.05,.05):
        gamma = i
        P1 = (df.rss.iloc[0]/abs(df.rss.iloc[0]))*abs(df.rss.iloc[0])**(1/gamma)
        Pj = (df.rss.iloc[1:]/abs(df.rss.iloc[1:]))*abs(df.rss.iloc[1:])**(1/gamma)
        A = np.array([2*(Pj**2*xj-P1**2*x1),2*(Pj**2*yj-P1**2*y1),P1**2-Pj**2]).transpose()
        b = np.array([Pj**2*(xj**2+yj**2)-P1**2*(x1**2+y1**2)]).transpose()
        theta = np.matmul(np.linalg.inv(np.matmul(A.transpose(),A)),np.matmul(A.transpose(),b))
        d = ((theta[0][0]-df.loc_x)**2+(theta[1][0]-df.loc_y)**2)**.5
        d1, di = d.iloc[0], d.iloc[1:]
        tot = sum((d1/di-Pj/P1)**2)
        if not optsum or (optsum and optsum > tot):
            optsum = tot
            optgam = gamma
            optx = theta[0][0]
            opty = theta[1][0]
    print("Optimal gamma: {} with total squared error: {}".format(optgam, optsum))
    print("Most accurate (x,y): {}".format((optx,opty)))

### Test on ground truth

In [4]:
findTrans(gtdf)

Optimal gamma: 2.8500000000000014 with total squared error: 237.30726756545076
Most accurate (x,y): (182.94467500271702, 143.9454353838656)


There is inaccuracy, but not terrible - 5 meters for x-axis, and .53 meters for y-axis. Now we will apply to find other APs.

### f8:cf:c5:97:e0:9e

In [5]:
findTrans(cfdf)

Optimal gamma: 5.0000000000000036 with total squared error: 267.7459390469134
Most accurate (x,y): (177.5140559802174, 143.5093015530926)


### ec:d0:9f:db:e8:1f

In [6]:
findTrans(d0df)

Optimal gamma: 3.100000000000002 with total squared error: 273.36559668022136
Most accurate (x,y): (175.69224090144775, 142.11826178777244)


### 80:e6:50:1b:a7:80

In [7]:
findTrans(e6df)

Optimal gamma: 5.0000000000000036 with total squared error: 1361.8511597252034
Most accurate (x,y): (175.15387195277967, 152.00867458428013)
